In [67]:
import pandas as pd
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import KFold

In [14]:
users_df = pd.read_csv('../data/raw/user.csv', index_col='user_id')
decision_df = pd.read_csv('../data/raw/user_decision.csv')
results_df = pd.read_csv('../data/raw/results.csv', index_col='id')
submit_df = pd.read_csv('../data/to_sibmit.csv', index_col='id')

In [3]:
len(decision_df['decision_id'].unique())

343

In [57]:
decision_ids = decision_df['decision_id'].value_counts().index

In [58]:
for decision_id in decision_ids:
    users_df[str(decision_id)] = users_df.apply(lambda user: ((decision_df['user_id']==user.name)&(decision_df['decision_id']==decision_id)).any(), axis = 1)

In [59]:
users_df.head()

,team_id,game_id,386,25,27,26,276,329,331,330,...,347,9,8,216,231,7,233,346,261,258
user_id,,,,,,,,,,,,,,,,,,,,,
10189,1664,235,True,True,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
10018,1690,237,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10580,1394,194,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10600,1908,262,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10112,1592,219,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False


In [68]:
users_known_df = results_df[[]].merge(users_df, left_index=True, right_index=True)
users_test_df = submit_df[[]].merge(users_df, left_index=True, right_index=True)

In [70]:
users_train_df, users_valid_df, y_train, y_valid = train_test_split(users_known_df, results_df['Analytical thinking'], test_size=0.25, random_state=42)

In [71]:
clf = GradientBoostingClassifier()
model = clf.fit(users_train_df, y_train)

In [72]:
pred = model.predict(users_valid_df)

In [73]:
recall_score(y_valid, pred, labels=[1.0,2.0,3.0,4.0,5.0,6.0], average='micro')

0.611764705882353

In [74]:
clf = GradientBoostingClassifier()
model_full = clf.fit(users_known_df, results_df['Analytical thinking'])

In [75]:
columns = ['Analytical thinking', 'Systemic thinking', 'Adaptability', 'Focus']
clf = GradientBoostingClassifier()
recall = 0
to_submit_df = submit_df[[]].copy()

for column in columns:
    users_train_df, users_valid_df, y_train, y_valid = train_test_split(users_known_df, results_df[column], test_size=0.3, random_state=42)
    model = clone(clf).fit(users_train_df, y_train)
    pred = model.predict(users_valid_df)
    recall += recall_score(y_valid, pred, labels=[1.0,2.0,3.0,4.0,5.0,6.0], average='micro')
    model_full = clone(clf).fit(users_known_df, results_df[column])
    to_submit_df[column] = model_full.predict(users_test_df)

print(recall/4.)
to_submit_df.to_csv('../data/submission/by_classifier.csv')

ValueError: Found input variables with inconsistent numbers of samples: [204, 170]